In [154]:
import random
import math
import numpy as np


cages = [
    ('*', 6, [(0, 0), (1, 0)]),
    ('+', 5, [(0, 1), (0, 2)]),
    ('/', 2, [(0, 3), (1, 3)]), 
    ('*', 8, [(2, 0), (3, 0), (3, 1)]),
    ('-', 1, [(1, 1), (2, 1)]),
    ('+', 3, [(1, 2), (2, 2)]),
    ('+', 7, [(2, 3), (3, 2), (3, 3)])
]

#cages = [
#    ('+', 7, [(0, 0), (0, 1)]),
#    ('*', 15, [(0, 2), (0, 3), (1, 2)]),
#    ('*', 10, [(0, 4), (1, 4), (2,4)]), 
#    ('*', 4, [(1, 0), (2, 0)]),
#    ('-', 2, [(1, 1), (2, 1)]),
#    ('+', 9, [(1, 3), (2, 3), (3,3)]),
#    ('-', 3, [(2, 2), (3, 2)]),
#    ('/', 2, [(3, 0), (3, 1)]),
#    ('*', 12, [(3, 4), (4, 4)]),
#    ('+', 7, [(4, 0), (4, 1)]),
#    ('-', 3, [(4, 2), (4, 3)])
#]

In [155]:
def generate_individual(grid_size, seed=None):
    random.seed(seed)
    while True:
        numbers = list(range(1, grid_size + 1))
        grid = np.zeros((grid_size, grid_size), dtype=int)

        for i in range(grid_size):
            random.shuffle(numbers)
            grid[i] = numbers[:]

        row_duplicates = any(len(np.unique(row)) < grid_size for row in grid)
        col_duplicates = any(len(np.unique(col)) < grid_size for col in grid.T)

        if not (row_duplicates or col_duplicates):
            break

    return grid


In [156]:
def create_population(population_size, grid_size):
    population = [generate_individual(grid_size, seed=i) for i in range(population_size)]
    return population

In [157]:
def calculate_cage_result(cage_values, operation):
    if operation == "+":
        return sum(cage_values)
    elif operation == "-":
        if len(cage_values) < 2:
            return 0  # Return 0 if there are not enough values for subtraction
        return abs(cage_values[0] - sum(cage_values[1:]))
    elif operation == "*":
        if len(cage_values) == 0:
            return 0  # Return 0 if there are no values for multiplication
        result = 1
        for value in cage_values:
            result *= value
        return result
    elif operation == "/":
        if len(cage_values) < 2:
            return 0  # Return 0 if there are not enough values for division
        cage_values = sorted(cage_values)
        result = cage_values[-1]
        for value in cage_values[:-1]:
            result /= value
        return math.floor(result)
    else:
        raise ValueError("Unsupported operation: {}".format(operation))


In [158]:
def check_uniqueness(solution, grid_size):
    fitness_penalty = 0

    try:
        grid = solution  # Assuming 'grid' is a key in your solution dictionary
        if grid is None or not isinstance(grid, np.ndarray) or grid.shape != (grid_size, grid_size):
            return grid_size  # Penalize with the maximum possible penalty

        for i in range(grid_size):
            row_values = set()
            col_values = set()
            for j in range(grid_size):
                value_row = grid[i, j]
                if value_row is None:
                    return grid_size  # Penalize with the maximum possible penalty

                if value_row in row_values:
                    fitness_penalty += 1
                row_values.add(value_row)

                value_col = grid[j, i]
                if value_col is None:
                    return grid_size  # Penalize with the maximum possible penalty

                if value_col in col_values:
                    fitness_penalty += 1
                col_values.add(value_col)

    except IndexError as e:
        print(f"IndexError: {e}")
        # Handle the error, possibly by returning a default penalty
        return grid_size

    return fitness_penalty

In [159]:
def evaluate_cage(solution, cages):
    fitness_penalty = 0
    
    grid = solution

    for operation, target_value, cells in cages:
        # Ensure the cells are valid indices for the grid
        valid_cells = [(i, j) for i, j in cells if 0 <= i < len(grid) and 0 <= j < len(grid[0])]

        # Extract the values in the cells corresponding to the current cage
        cage_values = [grid[i][j] for i, j in valid_cells]

        # Calculate the result of the cage operation using the extracted values
        cage_result = calculate_cage_result(cage_values, operation)

        # Add the absolute difference between the target value and the cage result to the penalty
        fitness_penalty += abs(target_value - cage_result)

    return int(fitness_penalty)


In [160]:
def evaluate_fitness(population, grid_size, cages):
    fitness_values = []
    for i in range(len(population)):
        uniqueness_penalty = check_uniqueness(population[i], grid_size)
        cage_penalty = evaluate_cage(population[i], cages)
        
        total_penalty = uniqueness_penalty + cage_penalty
        
        # Use the total penalty as the fitness score (negative to minimize)
        fitness_values.append(-total_penalty)
    
    return fitness_values


In [161]:
def selection(population, fitness_values):
    # Create a list to hold the selected population
    selected_population = []

    # Calculate total fitness
    total_fitness = sum(fitness_values)

    # Calculate selection probabilities
    selection_probabilities = [fitness / total_fitness for fitness in fitness_values]

    # Select individuals based on their probabilities
    for _ in range(len(population)):
        selected = False
        while not selected:
            # Randomly choose an individual index based on the probabilities
            selected_index = random.choices(range(len(population)), weights=selection_probabilities)[0]

            # Add the selected individual to the population
            selected_population.append(population[selected_index])
            selected = True

    return selected_population

In [162]:
def crossover(parent1, parent2):
    length = len(parent1)
    
    # No crossover is possible for short parents or if either parent is empty
    if length <= 2 or len(parent2) == 0:
        return parent1.copy(), parent2.copy()

    position = random.randint(1, length - 1)

    child1_list = list(parent1[:position]) + list(parent2[position:])
    child2_list = list(parent2[:position]) + list(parent1[position:])
    
    return child1_list, child2_list

In [163]:
def uniform_crossover(parent1, parent2, probability=0.5):
    # Ensure parents have the same length
    assert len(parent1) == len(parent2), "Parents must have the same length"

    # Create two child individuals by selecting genes with the specified probability
    child1 = [gene1 if random.random() < probability else gene2 for gene1, gene2 in zip(parent1, parent2)]
    child2 = [gene1 if random.random() < probability else gene2 for gene1, gene2 in zip(parent1, parent2)]

    return child1, child2


In [164]:
def mutate(solution, pm):
    # Mutate a solution with a probability pm
    mutated_solution = solution.copy()

    grid_size = len(solution)
    
    for i in range(grid_size):
        for j in range(grid_size):
            if random.random() < pm:
                mutated_solution[i][j] = np.random.randint(1, grid_size + 1)
    
    return mutated_solution

In [165]:
def swap_mutate(solution, pm):
    mutated_solution = solution.copy()
    grid_size = len(solution)
    
    for _ in range(grid_size):  # Perform multiple swap mutations
        if random.random() < pm:
            # Randomly choose two distinct indices (i, j) and (k, l)
            i, j = random.randint(0, grid_size - 1), random.randint(0, grid_size - 1)
            k, l = random.randint(0, grid_size - 1), random.randint(0, grid_size - 1)
            
            # Swap values at (i, j) and (k, l)
            mutated_solution[i][j], mutated_solution[k][l] = mutated_solution[k][l], mutated_solution[i][j]
    
    return mutated_solution

In [166]:
def kenken(pop_size, max_generations, grid_size, cages, pm):
    population = create_population(pop_size, grid_size)
    best_fitness_overall = None
    best_solution = None
    
    for i_gen in range(max_generations):
        fitness_values = evaluate_fitness(population, grid_size, cages)
        best_i = fitness_values.index(max(fitness_values))
        best_fitness = fitness_values[best_i]
        best_solution_gen = population[best_i]
        best_solution_gen = np.array(best_solution_gen)
        
        if best_fitness_overall is None or best_fitness > best_fitness_overall:
            best_fitness_overall = best_fitness
            best_solution = population[best_i]
        
        # Calculate and print the best fitness for the current population
        population_best_fitness = max(fitness_values)
        print(f'i_gen = {i_gen:06}   Best fitness in population: {-population_best_fitness:03}')
        print(best_solution_gen)
        if best_fitness == 0:
            print('Found optimal solution')
            break
        
        selected_pop = selection(population, fitness_values)
        children = []
        
        for i in range(0, len(selected_pop), 2):
            if i + 1 < len(selected_pop):
                child1, child2 = crossover(selected_pop[i], selected_pop[i + 1])
                # Apply mutation to children
                child1 = swap_mutate(child1, pm)
                child2 = swap_mutate(child2, pm)
                children.append(child1)
                children.append(child2)

        population = children  # Update the population with the crossover children
    
    print()
    print('Best solution:')
    print(best_solution)
    print('\r' + f' Best fitness={-best_fitness_overall:03}', end='')

    return best_solution  # Return the best_solution array


In [167]:
kenken(10, 1000, 4, cages, 0.9)

i_gen = 000000   Best fitness in population: 006
[[3 4 1 2]
 [1 2 3 4]
 [4 3 2 1]
 [2 1 4 3]]
i_gen = 000001   Best fitness in population: 013
[[3 2 4 1]
 [2 1 3 1]
 [1 4 2 3]
 [2 4 4 3]]
i_gen = 000002   Best fitness in population: 014
[[1 2 4 4]
 [3 3 2 3]
 [4 2 4 2]
 [1 2 3 4]]
i_gen = 000003   Best fitness in population: 013
[[4 4 2 4]
 [3 3 1 2]
 [1 3 2 4]
 [3 3 1 2]]
i_gen = 000004   Best fitness in population: 018
[[4 2 2 2]
 [3 1 4 3]
 [2 1 2 2]
 [1 3 3 2]]
i_gen = 000005   Best fitness in population: 023
[[3 2 4 2]
 [1 4 4 4]
 [2 3 4 4]
 [4 2 1 4]]
i_gen = 000006   Best fitness in population: 014
[[1 4 4 2]
 [4 3 2 1]
 [3 2 2 3]
 [1 2 2 4]]
i_gen = 000007   Best fitness in population: 012
[[2 4 4 1]
 [3 2 3 2]
 [1 2 4 2]
 [2 4 1 4]]
i_gen = 000008   Best fitness in population: 020
[[2 4 1 1]
 [4 1 4 2]
 [2 4 4 3]
 [2 3 3 4]]
i_gen = 000009   Best fitness in population: 018
[[3 3 1 2]
 [3 3 2 1]
 [1 2 4 4]
 [4 3 4 2]]
i_gen = 000010   Best fitness in population: 024
[[4 3 3 4]


array([[3, 4, 1, 2],
       [1, 2, 3, 4],
       [4, 3, 2, 1],
       [2, 1, 4, 3]])